<a href="https://colab.research.google.com/github/AlexeyTri/MLSeminars/blob/main/MLHSE/12_OPTUNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.5/210.5 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 5.5 MB/s eta 0:00:00


In [2]:
import optuna

def objective(trial):
    x = trial.suggest_float('x', -10,10)
    return (x-2)**2

study = optuna.create_study()
study.optimize(objective, n_trials=100)
study.best_params

[I 2023-03-01 08:18:05,231] A new study created in memory with name: no-name-b1df5e66-2009-43cf-be5d-91683a03a5c5
[I 2023-03-01 08:18:05,238] Trial 0 finished with value: 0.06891857400146599 and parameters: {'x': 2.2625234732389963}. Best is trial 0 with value: 0.06891857400146599.
[I 2023-03-01 08:18:05,241] Trial 1 finished with value: 73.02570216304424 and parameters: {'x': -6.545507718271879}. Best is trial 0 with value: 0.06891857400146599.
[I 2023-03-01 08:18:05,244] Trial 2 finished with value: 1.0725392719275242 and parameters: {'x': 3.0356347193521103}. Best is trial 0 with value: 0.06891857400146599.
[I 2023-03-01 08:18:05,248] Trial 3 finished with value: 0.01469620049603854 and parameters: {'x': 2.121227886626958}. Best is trial 3 with value: 0.01469620049603854.
[I 2023-03-01 08:18:05,251] Trial 4 finished with value: 51.83369991791502 and parameters: {'x': 9.1995624810064}. Best is trial 3 with value: 0.01469620049603854.
[I 2023-03-01 08:18:05,254] Trial 5 finished with 

{'x': 2.0250466206958153}

In [8]:
import sklearn
import sklearn.datasets
import sklearn.ensemble
import sklearn.svm

def objective(trial):
    iris = sklearn.datasets.load_iris()
    x, y = iris.data, iris.target

    classifier_name = trial.suggest_categorical('classifier', ['SVC', 'RandomForest'])
    if classifier_name == 'SVC':
        svc_c = trial.suggest_float('svc_c', 1e-10, 1e10, log=True)
        classifier_object = sklearn.svm.SVC(C=svc_c, gamma='auto')
    else: 
        rf_max_deph = trial.suggest_int("rf_max_deph", 1, 32, log=True)
        n_estimators = trial.suggest_int('n_estimators', 1, 10)
        classifier_object = sklearn.ensemble.RandomForestClassifier(n_estimators=n_estimators, max_depth=rf_max_deph)
    score = sklearn.model_selection.cross_val_score(classifier_object, x, y, n_jobs=-1, cv=3)

    accuracy = score.mean()
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)
    

[I 2023-03-01 08:54:18,096] A new study created in memory with name: no-name-6e991d60-891b-48b9-9950-418ffef7f01b
[I 2023-03-01 08:54:18,262] Trial 0 finished with value: 0.86 and parameters: {'classifier': 'RandomForest', 'rf_max_deph': 1, 'n_estimators': 8}. Best is trial 0 with value: 0.86.
[I 2023-03-01 08:54:18,306] Trial 1 finished with value: 0.94 and parameters: {'classifier': 'RandomForest', 'rf_max_deph': 10, 'n_estimators': 1}. Best is trial 1 with value: 0.94.
[I 2023-03-01 08:54:18,371] Trial 2 finished with value: 0.8866666666666667 and parameters: {'classifier': 'RandomForest', 'rf_max_deph': 1, 'n_estimators': 9}. Best is trial 1 with value: 0.94.
[I 2023-03-01 08:54:18,387] Trial 3 finished with value: 0.98 and parameters: {'classifier': 'SVC', 'svc_c': 11.463404431303923}. Best is trial 3 with value: 0.98.
[I 2023-03-01 08:54:18,404] Trial 4 finished with value: 0.32 and parameters: {'classifier': 'SVC', 'svc_c': 1.1307212664760164e-10}. Best is trial 3 with value: 0.

In [9]:
study.best_trial

FrozenTrial(number=11, state=TrialState.COMPLETE, values=[0.9866666666666667], datetime_start=datetime.datetime(2023, 3, 1, 8, 54, 18, 592348), datetime_complete=datetime.datetime(2023, 3, 1, 8, 54, 18, 610317), params={'classifier': 'SVC', 'svc_c': 4.564657501467379}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'classifier': CategoricalDistribution(choices=('SVC', 'RandomForest')), 'svc_c': FloatDistribution(high=10000000000.0, log=True, low=1e-10, step=None)}, trial_id=11, value=None)

In [10]:
study.best_params

{'classifier': 'SVC', 'svc_c': 4.564657501467379}

In [12]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 11.2 MB/s eta 0:00:00


In [15]:
import catboost as cb
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split 
import numpy as np

In [18]:
def objective(trial):
    data, target = load_breast_cancer(return_X_y=True)
    x_train, x_val, y_train, y_val = train_test_split(data, target, test_size=0.3)

    param ={
        'objective': trial.suggest_categorical('objective', ['Logloss', 'CrossEntropy']),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.01, 0.1),
        'depth': trial.suggest_int('depth', 1, 12),
        'boosting_type': trial.suggest_categorical('boosting_type', ['Ordered', 'Plain']),
        'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']),
        'used_ram_limit': '3gb'
        }
    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    gbm = cb.CatBoostClassifier(**param)
    gbm.fit(x_train, y_train, eval_set=[(x_val, y_val)], verbose=0, early_stopping_rounds=100)

    preds = gbm.predict(x_val)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(y_val, pred_labels)

    return accuracy        

In [19]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=600)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-03-01 09:28:32,673] A new study created in memory with name: no-name-cda73850-5ecf-4190-95c9-468e7458418c
[I 2023-03-01 09:28:37,181] Trial 0 finished with value: 0.9766081871345029 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.024328874550694025, 'depth': 4, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 0.5879456840827824}. Best is trial 0 with value: 0.9766081871345029.
[I 2023-03-01 09:28:43,932] Trial 1 finished with value: 0.9824561403508771 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.09729679483983218, 'depth': 6, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'subsample': 0.9393483170998828}. Best is trial 1 with value: 0.9824561403508771.
[I 2023-03-01 09:28:44,434] Trial 2 finished with value: 0.9824561403508771 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.062017939366631664, 'depth': 2, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'subsamp

Number of finished trials: 100
Best trial:
  Value: 0.9941520467836257
  Params: 
    objective: Logloss
    colsample_bylevel: 0.013267709284922921
    depth: 3
    boosting_type: Ordered
    bootstrap_type: Bernoulli
    subsample: 0.49149531583211153
